In [1]:
from langchain_community.tools import ShellTool
shell = ShellTool()

In [42]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain import hub 
from langchain.agents import create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults

In [43]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

In [54]:
tools = [ ShellTool()]
llm = OllamaFunctions(model="llama3.1").bind_tools(tools=tools)
prompt = hub.pull("hwchase17/react") 
agent = create_openai_functions_agent(llm,tools,prompt)

In [55]:
from langchain.agents import AgentExecutor

executor = AgentExecutor(
    tools=tools,
    agent=agent,
    verbose=True
)

executor.invoke({"input": "print the current working directory using pwd"})



> Entering new AgentExecutor chain...


KeyError: "Input to PromptTemplate is missing variables {'tool_names', 'tools'}.  Expected: ['agent_scratchpad', 'input', 'tool_names', 'tools'] Received: ['input', 'intermediate_steps', 'agent_scratchpad']"

In [18]:
from langchain_core.messages import BaseMessage 
from langchain_core.agents import AgentAction, AgentFinish
from typing import TypedDict, Annotated, List, Union
import operator

In [39]:
class AgentState(TypedDict): 
    input : str 
    chat_history : List[BaseMessage]
    agent_outcome : Union[AgentAction,AgentFinish,None]
    intermediate_steps : Annotated[List[tuple[AgentAction,str]],operator.add]

In [31]:
from langgraph.prebuilt.tool_executor import ToolExecutor
from langchain_community.tools import ShellTool

tool_executor = ToolExecutor([TavilySearchResults(max_results=1),ShellTool()])

# getting the outcome of an agent 
def run_agent(data): 
    agent_outcome = agent.invoke(data)
    return {"agent_outcome": agent_outcome}


def execute_tools(data):
    action = data["agent_outcome"]
    output = tool_executor.invoke(action)
    return {"intermediate_steps" :[(action,str(output))]}

def should_continue(data): 
    if isinstance(data['agent_outcome',AgentFinish]):
        return "END"
    return "CONT"

In [37]:
from langgraph.graph import END, StateGraph

flow = StateGraph(AgentState)

flow.add_node("agent",run_agent)
flow.add_node("action",execute_tools)
flow.set_entry_point("agent")

flow.add_conditional_edges(
    "agent", should_continue,{"CONT" : "action", "END" : END})

flow.add_edge("action","agent")
app = flow.compile()

In [38]:

inputs = {"input" : "execute the command that prints the current working directory", "chat_history": []}
for node in app.stream(inputs): 
    print(list(node.values()[0]))
    print("----------------------------------")

KeyError: ('agent_outcome', <class 'langchain_core.agents.AgentFinish'>)